In [1]:
import urllib.request
import ray
from threading import Thread
from itertools import chain
import os
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary


In [2]:

'''Para libros: https://www.amazon.es/dp/8434417219'''
ISBN10 = '8434417219' 
'''Para productos que llevan EAN, en informacion sobre el modelo '''
ean_list = ['8487497376498','8435484051026']

In [3]:
def bullet_points(all_bullets): 
    
    if len(all_bullets.split("\n")) == 8: 
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, bullet8= [str(e) for e in all_bullets.split("\n")]
    elif len(all_bullets.split("\n")) == 7: 
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7= [str(e) for e in all_bullets.split("\n")] 
        bullet8 = float('nan')
    elif len(all_bullets.split("\n")) == 6: 
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6 = [str(e) for e in all_bullets.split("\n")] 
        bullet7 = bullet8 = float('nan')
    elif len(all_bullets.split("\n")) == 5: 
        bullet1, bullet2, bullet3, bullet4, bullet5 = [str(e) for e in all_bullets.split("\n")] 
        bullet6 = bullet7 = bullet8 =  float('nan')
    elif len(all_bullets.split("\n")) == 4: 
        bullet1, bullet2, bullet3, bullet4 = [str(e) for e in all_bullets.split("\n")] 
        bullet5 = bullet6 = bullet7 = bullet8 = float('nan')
    elif len(all_bullets.split("\n")) == 3: 
        bullet1, bullet2, bullet3, = [str(e) for e in all_bullets.split("\n")]
        bullet4 = bullet5 = bullet6 = bullet7 = bullet8 =  float('nan')
    elif len(all_bullets.split("\n")) == 2: 
        bullet1, bullet2 = [str(e) for e in all_bullets.split("\n")]
        bullet3 = bullet4 = bullet5 = bullet6 = bullet7 = bullet8 =  float('nan')
    elif len(all_bullets.split("\n")) == 1: 
        bullet1 = [str(e) for e in all_bullets.split("\n")]
        bullet2 = bullet3 = bullet4 = bullet5 = bullet6 = bullet7 = bullet8 =  float('nan')
    else : 
        bullet1 = bullet2 = bullet3 = bullet4 = bullet5 = bullet6 = bullet7 = bullet8 =  float('nan') 
        
    return bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, bullet8  



def amazon(ean_list):
    
    amazon_info = pd.DataFrame()
    Detalles_tecnicos_t = pd.DataFrame()
    Detalles_usuarios_amazon_t = pd.DataFrame()
    
    for ean in ean_list:
        
        os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
        os.environ['PATH']

        browser = webdriver.Firefox()
        browser.get('https://www.amazon.es')

        search = browser.find_element_by_xpath('//input[@id="twotabsearchtextbox"]')
        search.send_keys(ean)
        search.send_keys(Keys.ENTER)

        time.sleep(3)

        action = ActionChains(browser)
        click_image  = browser.find_element_by_xpath("(//div[@class='sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32'])[2]")
        action.move_to_element((click_image)).click().perform()

        time.sleep(3)

        title = browser.find_elements_by_xpath('//h1[@id="title"]')[0].text
        price = browser.find_elements_by_xpath('//span[@id="priceblock_ourprice"]')[0].text
        all_bullets = browser.find_elements_by_xpath('//div[@id="feature-bullets"]')[0].text
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, bullet8 = bullet_points(all_bullets)
        main_image = browser.find_element_by_xpath('//img[@id="landingImage"]').get_attribute('src')
        
        amazon_selenium = pd.DataFrame({'ean.value': [ean],
                                     'item_name.value': [title], 
                                     'list_price.value_with_tax': [price],
                                     'bullet_point.value': [bullet1], 
                                     'bullet_point#2.value': [bullet2],
                                     'bullet_point#3.value': [bullet3], 
                                     'bullet_point#4.value': [bullet4], 
                                     'bullet_point#5.value': [bullet5],
                                     'bullet_point#6.value': [bullet6],
                                     'bullet_point#7.value': [bullet7],  
                                     'main_image': [main_image]
                                    })
        
        browser.close()
        amazon_info = pd.concat([amazon_info, amazon_selenium], axis=1, sort=False)
      
    return amazon_info

In [4]:
def ebay(ean_list):
    
    ebay_info = pd.DataFrame()
   
    for ean in ean_list:

        os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
        os.environ['PATH']

        browser = webdriver.Firefox()
        browser.get('https://www.ebay.es/')

        search = browser.find_element_by_xpath('//input[@class="gh-tb ui-autocomplete-input"]')
        search.send_keys(ean)
        search.send_keys(Keys.ENTER)

        time.sleep(3)

        action = ActionChains(browser)
        click_image  = browser.find_element_by_xpath("(//img[@class='img'])[1]")
        action.move_to_element((click_image)).click().perform()

        time.sleep(3)

        title = browser.find_elements_by_xpath('//h1[@id="itemTitle"]')[0].text
        price = browser.find_elements_by_xpath('//span[@id="prcIsum"]')[0].text
        main_image = browser.find_element_by_xpath('//img[@id="icImg"]').get_attribute('src')


        ebay_selenium = pd.DataFrame({'ean.value': [ean],
                                     'item_name.value': [title], 
                                     'list_price.value_with_tax': [price],
                                     'main_image': [main_image]
                                    })
        
        browser.close()
        ebay_info = pd.concat([ebay_info, ebay_selenium], axis=1, sort=False)

            
    return  ebay_info

In [5]:

ray.init()

# Define functions you want to execute in parallel using 
# the ray.remote decorator.
@ray.remote
def amazon(ean_list):
    
    amazon_info = pd.DataFrame()
    Detalles_tecnicos_t = pd.DataFrame()
    Detalles_usuarios_amazon_t = pd.DataFrame()
    
    for ean in ean_list:
        
        os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
        os.environ['PATH']

        browser = webdriver.Firefox()
        browser.get('https://www.amazon.es')

        search = browser.find_element_by_xpath('//input[@id="twotabsearchtextbox"]')
        search.send_keys(ean)
        search.send_keys(Keys.ENTER)

        time.sleep(3)

        action = ActionChains(browser)
        click_image  = browser.find_element_by_xpath("(//div[@class='sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32'])[2]")
        action.move_to_element((click_image)).click().perform()

        time.sleep(3)

        title = browser.find_elements_by_xpath('//h1[@id="title"]')[0].text
        price = browser.find_elements_by_xpath('//span[@id="priceblock_ourprice"]')[0].text
        all_bullets = browser.find_elements_by_xpath('//div[@id="feature-bullets"]')[0].text
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, bullet8 = bullet_points(all_bullets)
        main_image = browser.find_element_by_xpath('//img[@id="landingImage"]').get_attribute('src')
        
        amazon_selenium = pd.DataFrame({'ean.value': [ean],
                                     'item_name.value': [title], 
                                     'list_price.value_with_tax': [price],
                                     'bullet_point.value': [bullet1], 
                                     'bullet_point#2.value': [bullet2],
                                     'bullet_point#3.value': [bullet3], 
                                     'bullet_point#4.value': [bullet4], 
                                     'bullet_point#5.value': [bullet5],
                                     'bullet_point#6.value': [bullet6],
                                     'bullet_point#7.value': [bullet7],  
                                     'main_image': [main_image]
                                    })
        
        browser.close()
        amazon_info = pd.concat([amazon_info, amazon_selenium], axis=1, sort=False)
      
    return amazon_info

@ray.remote

def ebay(ean_list):
    
    ebay_info = pd.DataFrame()
   
    for ean in ean_list:

        os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
        os.environ['PATH']

        browser = webdriver.Firefox()
        browser.get('https://www.ebay.es/')

        search = browser.find_element_by_xpath('//input[@class="gh-tb ui-autocomplete-input"]')
        search.send_keys(ean)
        search.send_keys(Keys.ENTER)

        time.sleep(3)

        action = ActionChains(browser)
        click_image  = browser.find_element_by_xpath("(//img[@class='img'])[1]")
        action.move_to_element((click_image)).click().perform()

        time.sleep(3)

        title = browser.find_elements_by_xpath('//h1[@id="itemTitle"]')[0].text
        price = browser.find_elements_by_xpath('//span[@id="prcIsum"]')[0].text
        main_image = browser.find_element_by_xpath('//img[@id="icImg"]').get_attribute('src')


        ebay_selenium = pd.DataFrame({'ean.value': [ean],
                                     'item_name.value': [title], 
                                     'list_price.value_with_tax': [price],
                                     'main_image': [main_image]
                                    })
        
        browser.close()
        ebay_info = pd.concat([ebay_info, ebay_selenium], axis=1, sort=False)

    return  ebay_info


# Execute func1 and func2 in parallel.

ret_id1 = amazon.remote(ean_list)
ret_id2 = ebay.remote(ean_list)

amazon_info, ebay_info = ray.get([ret_id1, ret_id2])

2020-03-23 10:45:45,170	INFO resource_spec.py:212 -- Starting Ray with 1.22 GiB memory available for workers and up to 0.63 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-23 10:45:46,162	INFO services.py:1078 -- View the Ray dashboard at localhost:8265
2020-03-23 10:45:56,604	WARNING worker.py:1058 -- The actor or task with ID 45b95b1c8bd3a9c4ffffffff0100 is pending and cannot currently be scheduled. It requires {CPU: 1.000000} for execution and {CPU: 1.000000} for placement, but this node only has remaining {CPU: 8.000000}, {object_store_memory: 0.390625 GiB}, {node:192.168.1.42: 1.000000}, {memory: 1.220703 GiB}. In total there are 2 pending tasks and 0 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected 

In [9]:
print(ebay_info)

None
